In [1]:
import torch
assert torch.__version__>='1.2.0', 'Expect PyTorch>=1.2.0 but get {}'.format(torch.__version__)
from torch import nn
import torch.nn.functional as F

import numpy as np

import time
import os
import sys
import pickle

imp_dir = '../Implementations'
sys.path.insert(1, imp_dir)
data_dir = '../Data/criteo'
sys.path.insert(1, data_dir)

import logging
import importlib
importlib.reload(logging)

logging.basicConfig(stream=sys.stdout, format='%(asctime)s %(levelname)-6s %(message)s', level=logging.INFO, datefmt='%H:%M:%S')
logger = logging.getLogger(__name__)

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info('Device in Use: {}'.format(DEVICE))
torch.cuda.empty_cache()
t = torch.cuda.get_device_properties(DEVICE).total_memory/1024**3
c = torch.cuda.memory_cached(DEVICE)/1024**3
a = torch.cuda.memory_allocated(DEVICE)/1024**3
logger.info('CUDA Memory: Total {:.2f} GB, Cached {:.2f} GB, Allocated {:.2f} GB'.format(t,c,a))

00:10:18 INFO   Device in Use: cuda
00:10:18 INFO   CUDA Memory: Total 11.17 GB, Cached 0.00 GB, Allocated 0.00 GB


## Prepare Data

### Load Dict

In [3]:
embedding_map_dict_pkl_path = os.path.join(data_dir, 'criteo_feature_dict_artifact/categorical_feature_map_dict.pkl')
with open(embedding_map_dict_pkl_path, 'rb') as f:
    embedding_map_dict = pickle.load(f)

### List all available files

In [4]:
np_artifact_dir = os.path.join(data_dir, 'criteo_train_numpy_artifact')
index_artifact = sorted(list(filter(lambda x: x.split('-')[1]=='index', os.listdir(np_artifact_dir))), key = lambda x: int(x.split('.')[0].split('-')[-1]))
value_artifact = sorted(list(filter(lambda x: x.split('-')[1]=='value', os.listdir(np_artifact_dir))), key = lambda x: int(x.split('.')[0].split('-')[-1]))
label_artifact = sorted(list(filter(lambda x: x.split('-')[1]=='label', os.listdir(np_artifact_dir))), key = lambda x: int(x.split('.')[0].split('-')[-1]))

In [ ]:
start = time.time()

train_data = (
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in index_artifact[:10]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in value_artifact[:10]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in label_artifact[:10]]),
)

logger.info('Training data loaded after {:.2f}s'.format(time.time()-start))

start = time.time()

test_data = (
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in index_artifact[10:]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in value_artifact[10:]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in label_artifact[10:]]),
)

logger.info('Test data loaded after {:.2f}s'.format(time.time()-start))

## Create Model

In [23]:
import execution
importlib.reload(execution)
import DCN_BinClf_Torch 
importlib.reload(DCN_BinClf_Torch)

<module 'DCN_BinClf_Torch' from '../Implementations/DCN_BinClf_Torch.py'>

In [24]:
DCN = DCN_BinClf_Torch.DCN_Layer(len(embedding_map_dict)+60,
                                 20,
                                 26,
                                 13,
                                 [1024 for _ in range(2)],
                                 [0.5 for _ in range(3)],
                                 6).to(DEVICE)

## Sample Run

In [25]:
cwd = os.getcwd()
checkpoint_dir = os.path.join(cwd, 'DCN_artifact')
checkpoint_prefix = 'DCN'

In [26]:
execution.train_model_separate_inp(DCN, 
                                   train_data, 
                                   train_data, 
                                   F.binary_cross_entropy_with_logits, 
                                   torch.optim.Adam(DCN.parameters()), 
                                   DEVICE, 
                                   checkpoint_dir, 
                                   checkpoint_prefix
                                  )

00:06:59 INFO   Epoch 1/5 - Batch 1/6400 Done - Train Loss: 0.708079, Val Loss: 1.322342
00:07:00 INFO   Epoch 1/5 - Batch 2/6400 Done - Train Loss: 0.673500, Val Loss: 0.687095
00:07:00 INFO   Epoch 1/5 - Batch 3/6400 Done - Train Loss: 0.639062, Val Loss: 0.655121
00:07:00 INFO   Epoch 1/5 - Batch 4/6400 Done - Train Loss: 0.646194, Val Loss: 0.633870
00:07:01 INFO   Epoch 1/5 - Batch 5/6400 Done - Train Loss: 0.639051, Val Loss: 0.625099
00:07:01 INFO   Epoch 1/5 - Batch 6/6400 Done - Train Loss: 0.636626, Val Loss: 0.613462
00:07:01 INFO   Epoch 1/5 - Batch 7/6400 Done - Train Loss: 0.635699, Val Loss: 0.603782
00:07:01 INFO   Epoch 1/5 - Batch 8/6400 Done - Train Loss: 0.632878, Val Loss: 0.594425
00:07:02 INFO   Epoch 1/5 - Batch 9/6400 Done - Train Loss: 0.632071, Val Loss: 0.592475
00:07:02 INFO   Epoch 1/5 - Batch 10/6400 Done - Train Loss: 0.628984, Val Loss: 0.605773
00:07:02 INFO   Epoch 1/5 - Batch 11/6400 Done - Train Loss: 0.619818, Val Loss: 0.584357
00:07:03 INFO   Epo

KeyboardInterrupt: 

## Model Performance

In [24]:
os.listdir(checkpoint_dir)

['DeepFM_1', 'DeepFM_2', 'DeepFM_5', 'DeepFM_4', 'DeepFM_3']

In [25]:
DCN = DCN_BinClf_Torch.DCN_Layer(len(embedding_map_dict)+60,
                                 20,
                                 26,
                                 13,
                                 [1024 for _ in range(2)],
                                 [0.5 for _ in range(3)],
                                 6).to(DEVICE)
model.load_state_dict(torch.load(os.path.join(checkpoint_dir, 'DCN_2')))
model = model.to(DEVICE)

In [26]:
logger.info('Model ROC Score: {:.6f}'.format(execution.get_roc_auc_score_separate_inp(model, train_data, DEVICE)))

20:29:43 INFO   Model ROC Score: 0.819064


In [27]:
logger.info('Model ROC Score: {:.6f}'.format(execution.get_roc_auc_score_separate_inp(model, test_data, DEVICE)))

20:30:42 INFO   Model ROC Score: 0.801416
